In [1]:
import pandas as pd
from geopy.distance import geodesic
import numpy as np
import matplotlib.pyplot as plt
import pickle
from datetime import datetime, timedelta

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.cluster import KMeans
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.utils import resample
from sklearn.svm import SVC


clean data. make rolling window

In [83]:
fd = pd.read_pickle('data/on_faults.pkl')

In [84]:
fd['EventTimeStamp'] = pd.to_datetime(fd['EventTimeStamp'])

In [85]:
fd = fd.loc[fd['active'] == True]

In [86]:
fd = fd.replace({'unknown': '0'})

In [88]:
fd_time = fd['EventTimeStamp']

In [81]:
fd

,spn,AcceleratorPedal,EngineRpm,EquipmentID,EngineOilTemperature,EngineLoad,EngineOilPressure,fmi
0,111,0,0,1439,96,11,0,17
1,629,0,0,1439,0,0,0,12
3,1807,0,0,1369,0,0,0,2
5,111,48,1514,1417,196,30,38,17
6,111,82,1711,1597,210,80,39,17
...,...,...,...,...,...,...,...,...
1057451,96,0,597,1936,197,11,22,3
1057454,111,0,600,1886,207,11,19,18
1057457,1569,100,1310,1994,211,51,37,31
1057458,3216,0,1340,1850,212,62,41,10


In [91]:
fd = fd[['spn', 'AcceleratorPedal','EngineRpm','EquipmentID', 'EngineOilTemperature',
                                             'EngineLoad', 'EngineOilPressure','fmi']]

# Convert the columns to numeric data types
fd = fd.apply(pd.to_numeric, errors='coerce')

# Fill NaN values with 0 and convert to integers
fd = fd.fillna(0).astype(int)

In [96]:
fd = pd.merge(fd_time, fd, left_index=True, right_index=True)

In [97]:
fd

,EventTimeStamp,spn,AcceleratorPedal,EngineRpm,EquipmentID,EngineOilTemperature,EngineLoad,EngineOilPressure,fmi
0,2015-02-21 10:47:13,111,0,0,1439,96,11,0,17
1,2015-02-21 11:34:34,629,0,0,1439,0,0,0,12
3,2015-02-21 11:35:33,1807,0,0,1369,0,0,0,2
5,2015-02-21 11:40:22,111,48,1514,1417,196,30,38,17
6,2015-02-21 11:40:52,111,82,1711,1597,210,80,39,17
...,...,...,...,...,...,...,...,...,...
1057451,2020-03-06 13:12:43,96,0,597,1936,197,11,22,3
1057454,2020-03-06 13:42:48,111,0,600,1886,207,11,19,18
1057457,2020-03-06 14:04:23,1569,100,1310,1994,211,51,37,31
1057458,2020-03-06 14:13:38,3216,0,1340,1850,212,62,41,10


In [98]:
fd_dum =pd.get_dummies(data=fd.loc[:, ['EventTimeStamp',
                                       'spn', 
                                       'AcceleratorPedal',
                                       'EngineRpm',
                                       'EquipmentID', 
                                       'EngineOilTemperature',
                                       'EngineLoad', 
                                       'EngineOilPressure',
                                       'fmi']], 
                           columns=['spn']).sort_values(['EquipmentID', 'EventTimeStamp']).reset_index(drop = True)
                                                            

In [99]:
fd_rolling = (fd_dum
              .groupby('EquipmentID')
                        .rolling(window = '1D', on = 'EventTimeStamp')
                        .sum()
                 )
                        

In [100]:
fd_rolling

EventTimeStamp  AcceleratorPedal  EngineLoad   
EquipmentID                                                            
0           0      2015-02-24 13:45:06               0.0        83.0  \
            1      2015-02-24 15:31:17               0.0       136.0   
            2      2015-02-25 06:08:43              21.0       148.0   
            3      2015-02-26 13:12:11               0.0        11.0   
            4      2018-06-12 14:12:40               0.0        66.0   
...                                ...               ...         ...   
2377        548843 2020-02-28 04:56:18               0.0        25.0   
            548844 2020-03-03 07:47:01               0.0        37.0   
            548845 2020-03-06 14:14:13               1.0         0.0   
2380        548846 2020-03-04 10:06:10               0.0        12.0   
2381        548847 2020-03-06 11:56:38               0.0        16.0   

                    EngineOilPressure  EngineOilTemperature  EngineRpm   fmi   
EquipmentID                                                                    
0           0                    42.0                 222.0     1333.0  17.0  \
            1                    84.0                 444.0     2685.0  35.0   
            2                   124.0                 604.0     3630.0  37.0   
            3                    38.0                 148.0      595.0   9.0   
            4                    31.0                 179.0      803.0  18.0   
...                               ...                   ...        ...   ...   
2377        548843               45.0                  69.0      600.0  18.0   
            548844               43.0                  77.0      593.0  18.0   
            548845               27.0                 221.0      863.0  18.0   
2380        548846               30.0                 165.0      599.0  17.0   
2381        548847               27.0                 178.0      605.0  18.0   

                    spn_0  spn_100  spn_101  ...  spn_938  spn_939  spn_94   
EquipmentID                                  ...                             
0           0         0.0      0.0      0.0  ...      0.0      0.0     0.0  \
            1         0.0      0.0      0.0  ...      0.0      0.0     0.0   
            2         0.0      0.0      0.0  ...      0.0      0.0     0.0   
            3         0.0      0.0      0.0  ...      0.0      0.0     0.0   
            4         0.0      0.0      0.0  ...      0.0      0.0     0.0   
...                   ...      ...      ...  ...      ...      ...     ...   
2377        548843    0.0      0.0      0.0  ...      0.0      0.0     0.0   
            548844    0.0      0.0      0.0  ...      0.0      0.0     0.0   
            548845    0.0      0.0      0.0  ...      0.0      0.0     0.0   
2380        548846    0.0      0.0      0.0  ...      0.0      0.0     0.0   
2381        548847    0.0      0.0      0.0  ...      0.0      0.0     0.0   

                    spn_940  spn_941  spn_95  spn_96  spn_97  spn_976  spn_98  
EquipmentID                                                                    
0           0           0.0      0.0     0.0     0.0     0.0      0.0     0.0  
            1           0.0      0.0     0.0     0.0     0.0      0.0     0.0  
            2           0.0      0.0     0.0     0.0     0.0      0.0     0.0  
            3           0.0      0.0     0.0     0.0     0.0      0.0     0.0  
            4           0.0      0.0     0.0     0.0     0.0      0.0     0.0  
...                     ...      ...     ...     ...     ...      ...     ...  
2377        548843      0.0      0.0     0.0     0.0     0.0      0.0     0.0  
            548844      0.0      0.0     0.0     0.0     0.0      0.0     0.0  
            548845      0.0      0.0     0.0     0.0     0.0      0.0     0.0  
2380        548846      0.0      0.0     0.0     0.0     0.0      0.0     0.0  
2381        548847      0.0      0.0     0.0     0.0     0.0      0.0  

In [101]:
fd_model  = fd_rolling.loc[~(fd_rolling ['spn_5246'] > 1)]

In [102]:
fd_model

EventTimeStamp  AcceleratorPedal  EngineLoad   
EquipmentID                                                            
0           0      2015-02-24 13:45:06               0.0        83.0  \
            1      2015-02-24 15:31:17               0.0       136.0   
            2      2015-02-25 06:08:43              21.0       148.0   
            3      2015-02-26 13:12:11               0.0        11.0   
            4      2018-06-12 14:12:40               0.0        66.0   
...                                ...               ...         ...   
2377        548843 2020-02-28 04:56:18               0.0        25.0   
            548844 2020-03-03 07:47:01               0.0        37.0   
            548845 2020-03-06 14:14:13               1.0         0.0   
2380        548846 2020-03-04 10:06:10               0.0        12.0   
2381        548847 2020-03-06 11:56:38               0.0        16.0   

                    EngineOilPressure  EngineOilTemperature  EngineRpm   fmi   
EquipmentID                                                                    
0           0                    42.0                 222.0     1333.0  17.0  \
            1                    84.0                 444.0     2685.0  35.0   
            2                   124.0                 604.0     3630.0  37.0   
            3                    38.0                 148.0      595.0   9.0   
            4                    31.0                 179.0      803.0  18.0   
...                               ...                   ...        ...   ...   
2377        548843               45.0                  69.0      600.0  18.0   
            548844               43.0                  77.0      593.0  18.0   
            548845               27.0                 221.0      863.0  18.0   
2380        548846               30.0                 165.0      599.0  17.0   
2381        548847               27.0                 178.0      605.0  18.0   

                    spn_0  spn_100  spn_101  ...  spn_938  spn_939  spn_94   
EquipmentID                                  ...                             
0           0         0.0      0.0      0.0  ...      0.0      0.0     0.0  \
            1         0.0      0.0      0.0  ...      0.0      0.0     0.0   
            2         0.0      0.0      0.0  ...      0.0      0.0     0.0   
            3         0.0      0.0      0.0  ...      0.0      0.0     0.0   
            4         0.0      0.0      0.0  ...      0.0      0.0     0.0   
...                   ...      ...      ...  ...      ...      ...     ...   
2377        548843    0.0      0.0      0.0  ...      0.0      0.0     0.0   
            548844    0.0      0.0      0.0  ...      0.0      0.0     0.0   
            548845    0.0      0.0      0.0  ...      0.0      0.0     0.0   
2380        548846    0.0      0.0      0.0  ...      0.0      0.0     0.0   
2381        548847    0.0      0.0      0.0  ...      0.0      0.0     0.0   

                    spn_940  spn_941  spn_95  spn_96  spn_97  spn_976  spn_98  
EquipmentID                                                                    
0           0           0.0      0.0     0.0     0.0     0.0      0.0     0.0  
            1           0.0      0.0     0.0     0.0     0.0      0.0     0.0  
            2           0.0      0.0     0.0     0.0     0.0      0.0     0.0  
            3           0.0      0.0     0.0     0.0     0.0      0.0     0.0  
            4           0.0      0.0     0.0     0.0     0.0      0.0     0.0  
...                     ...      ...     ...     ...     ...      ...     ...  
2377        548843      0.0      0.0     0.0     0.0     0.0      0.0     0.0  
            548844      0.0      0.0     0.0     0.0     0.0      0.0     0.0  
            548845      0.0      0.0     0.0     0.0     0.0      0.0     0.0  
2380        548846      0.0      0.0     0.0     0.0     0.0      0.0     0.0  
2381        548847      0.0      0.0     0.0     0.0     0.0      0.0  

model 1: RandomForestClassifier

In [103]:
# Separate majority and minority classes
df_majority = fd_model[fd_model.spn_5246==0]
df_minority = fd_model[fd_model.spn_5246==1]
 
# Downsample majority class
df_majority_downsampled = resample(df_majority, 
                                 replace=False,    # sample without replacement
                                 n_samples=2000,     # to match minority class
                                 random_state=200) # reproducible results
 
# Combine minority class with downsampled majority class
df_downsampled = pd.concat([df_majority_downsampled, df_minority])
 
# Display new class counts
df_downsampled.spn_5246.value_counts()

spn_5246
0.0    2000
1.0    1218
Name: count, dtype: int64

In [104]:
X = df_downsampled.drop(columns=['spn_5246', 'EventTimeStamp'])

y = df_downsampled['spn_5246']

X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=.25, random_state = 321)

In [105]:
scaler = StandardScaler()
imputer = SimpleImputer()
rfc = RandomForestClassifier  # LogisticRegression

In [106]:
steps = [("imputer", imputer), 
         ("scaler", scaler),
         ("rfc", RandomForestClassifier(n_estimators=100, max_depth = 100
                    )
         )
         
        ]


pipeline = Pipeline(steps = steps)

In [107]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('imputer', SimpleImputer()), ('scaler', StandardScaler()),
                ('rfc', RandomForestClassifier(max_depth=100))])

In [108]:
print(classification_report(y_test, pipeline.predict(X_test)))

              precision    recall  f1-score   support

         0.0       0.97      0.97      0.97       505
         1.0       0.94      0.94      0.94       300

    accuracy                           0.96       805
   macro avg       0.95      0.95      0.95       805
weighted avg       0.96      0.96      0.96       805



In [109]:
print(confusion_matrix(y_test, pipeline.predict(X_test)))

[[488  17]
 [ 17 283]]


In [110]:
X2 = fd_model.drop(columns=['spn_5246', 'EventTimeStamp']) # make df equipmentID , stratiy on variable of trucks w/ derate

y2 = fd_model['spn_5246'] # equipmentID

X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=.25, random_state = 321)

In [111]:
scaler = StandardScaler()
imputer = SimpleImputer()
rfc = RandomForestClassifier 

In [112]:
steps = [("imputer", imputer), 
         ("scaler", scaler),
         ("rfc", RandomForestClassifier(n_estimators=100, max_depth = 200
                    )
         )
         
        ]


pipeline = Pipeline(steps = steps)

In [113]:
pipeline.fit(X2_train, y2_train)

Pipeline(steps=[('imputer', SimpleImputer()), ('scaler', StandardScaler()),
                ('rfc', RandomForestClassifier(max_depth=200))])

In [114]:
print(classification_report(y2_test, pipeline.predict(X2_test)))

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00    136828
         1.0       0.90      0.47      0.62       286

    accuracy                           1.00    137114
   macro avg       0.95      0.73      0.81    137114
weighted avg       1.00      1.00      1.00    137114



In [115]:
print(confusion_matrix(y2_test, pipeline.predict(X2_test)))


[[136813     15]
 [   152    134]]


In [49]:
#scores = cross_val_score(rfc, X2, y2, cv=5, scoring='f1')

In [ ]:
# true neg = top left 
# false pos = top right
# false neg = bottom left
# true pos = bottom right 

In [ ]:
#full_derates_seasons = full_derates_seasons[['EventTimeStamp',spn', 'AcceleratorPedal','EngineRpm','EquipmentID', 'EngineOilTemperature',
                                             #'EngineLoad', 'EngineOilPressure','fmi']]

# Convert the columns to numeric data types
#full_derates_seasons = full_derates_seasons.apply(pd.to_numeric, errors='coerce')

# Fill NaN values with 0 and convert to integers
#full_derates_seasons = full_derates_seasons.fillna(0).astype(int)

In [ ]:
#full_derates_seasons=full_derates_seasons.set_index('EventTimeStamp').sort_index(ascending = False)